In [1]:
import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall


/home/yapayzeka/ahsen_bulbul/ahsen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct
from qdrant_client.http import models
import uuid
from tqdm import tqdm
import json
from typing import List, Dict, Any
import re
from dotenv import load_dotenv
import os


In [3]:

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

qdrant_url = os.getenv("QDRANT_URL")
api_key = os.getenv("QDRANT_API_KEY")
collection_name = os.getenv("QDRANT_COLLECTION")

In [4]:
class LegalDocumentVectorDB:
    def __init__(self, qdrant_url: str, api_key: str, collection_name: str = "hukuki kararlar"):
        """
        Hukuki belgeler için vector database sınıfı
        
        Args:
            qdrant_url: Qdrant sunucu URL'i
            api_key: Qdrant API anahtarı  
            collection_name: Collection adı
        """
        self.client = QdrantClient(
            url=qdrant_url,
            api_key=api_key,
            timeout=60
        )
        self.collection_name = collection_name
        
        # Türkçe için optimize edilmiş multilingual model
        print("Sentence Transformer modeli yükleniyor...")
        self.model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        print("Model başarıyla yüklendi!")
        
        # Vector boyutunu al
        self.vector_size = self.model.get_sentence_embedding_dimension()
        print(f"Vector boyutu: {self.vector_size}")

    def create_collection(self, recreate: bool = False):
        """Collection oluştur"""
        try:
            if recreate:
                self.client.delete_collection(collection_name=self.collection_name)
                print(f"Eski collection '{self.collection_name}' silindi.")
        except:
            pass
        
        try:
            self.client.create_collection(
                collection_name=self.collection_name,
                vectors_config=VectorParams(
                    size=self.vector_size, 
                    distance=Distance.COSINE
                )
            )
            print(f"Collection '{self.collection_name}' oluşturuldu.")
        except Exception as e:
            if "already exists" in str(e).lower():
                print(f"Collection '{self.collection_name}' zaten mevcut.")
            else:
                raise e

    def clean_text(self, text: str) -> str:
        """Metni temizle"""
        if pd.isna(text):
            return ""
        
        # Encoding sorunlarını düzelt
        text = str(text)
        replacements = {
            'Ã¤': 'ä', 'Ã¶': 'ö', 'Ã¼': 'ü', 'ÃŸ': 'ß',
            'Ã‡': 'Ç', 'Ä±': 'ı', 'Ä°': 'İ', 'ÅŸ': 'ş',
            'Ä\x9f': 'ğ', 'Ã§': 'ç', 'Ã¶': 'ö', 'Ã¼': 'ü'
        }
        
        for old, new in replacements.items():
            text = text.replace(old, new)
        
        # Fazla boşlukları temizle
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text

    def process_csv(self, csv_path: str) -> pd.DataFrame:
        """CSV dosyasını işle"""
        print(f"CSV dosyası okunuyor: {csv_path}")
        
        # Dosya varlığını kontrol et
        import os
        if not os.path.exists(csv_path):
            print(f"❌ HATA: Dosya bulunamadı: {csv_path}")
            return None
        
        try:
            # Encoding denemesi
            try:
                df = pd.read_csv(csv_path, encoding='utf-8')
                print("✅ UTF-8 encoding ile başarıyla okundu")
            except UnicodeDecodeError:
                df = pd.read_csv(csv_path, encoding='latin-1')
                print("✅ Latin-1 encoding ile başarıyla okundu")
        except Exception as e:
            print(f"❌ CSV okuma hatası: {e}")
            return None
        
        print(f"Toplam satır sayısı: {len(df)}")
        print(f"Sütunlar: {df.columns.tolist()}")
        
        # Boş chunk_text'leri filtrele
        initial_count = len(df)
        df = df.dropna(subset=['chunk_text'])
        df = df[df['chunk_text'].str.strip() != '']
        final_count = len(df)
        
        print(f"Boş metin filtrelemesi: {initial_count} -> {final_count}")
        
        # Metinleri temizle
        df['chunk_text_clean'] = df['chunk_text'].apply(self.clean_text)
        
        return df

    def create_embeddings_batch(self, texts: List[str], batch_size: int = 32) -> List[List[float]]:
        """Metinleri batch halinde embedding'e çevir"""
        embeddings = []
        
        for i in tqdm(range(0, len(texts), batch_size), desc="Embedding oluşturuluyor"):
            batch = texts[i:i+batch_size]
            batch_embeddings = self.model.encode(
                batch, 
                convert_to_numpy=True,
                show_progress_bar=False,
                normalize_embeddings=True  # Cosine similarity için normalize et
            )
            embeddings.extend(batch_embeddings.tolist())
            print(embeddings)
        
        return embeddings

    def upload_to_qdrant(self, df: pd.DataFrame, batch_size: int = 100):
        """DataFrame'i Qdrant'a yükle"""
        print("Qdrant'a yükleme başlıyor...")
        
        # Embeddings oluştur
        texts = df['chunk_text_clean'].tolist()
        embeddings = self.create_embeddings_batch(texts, batch_size=32)
        
        # Points oluştur
        points = []
        for idx, (_, row) in enumerate(df.iterrows()):
            payload = {
                "document_id": str(row['_id']),
                "location": str(row['location']),
                "esas_no": str(row['esasNo']),
                "karar_no": str(row['kararNo']),
                "dates": str(row['extractedDates']),
                "daire": str(row['daire']),
                "mahkeme": str(row['mahkeme']),
                "karar_turu": str(row['karar_turu']),
                "chunk_id": str(row['chunk_id']),
                "chunk_index": int(row['chunk_index']),
                "total_chunks": int(row['total_chunks']),
                "chunk_text": str(row['chunk_text_clean']),
                "chunk_length": int(row['chunk_length'])
            }
            
            point = PointStruct(
                id=str(uuid.uuid4()),
                vector=embeddings[idx],
                payload=payload
            )
            points.append(point)
        
        # Batch halinde yükle
        for i in tqdm(range(0, len(points), batch_size), desc="Qdrant'a yükleniyor"):
            batch_points = points[i:i+batch_size]
            
            try:
                self.client.upsert(
                    collection_name=self.collection_name,
                    points=batch_points
                )
            except Exception as e:
                print(f"Yükleme hatası (batch {i//batch_size + 1}): {e}")
                continue
        
        print(f"Toplam {len(points)} doküman başarıyla yüklendi!")

    def search(self, query: str, limit: int = 5, score_threshold: float = 0.5) -> List[Dict[str, Any]]:
        """Semantik arama yap"""
        print(f"Arama yapılıyor: '{query}'")
        
        # Query embedding'i oluştur
        query_embedding = self.model.encode(
            [query], 
            convert_to_numpy=True,
            normalize_embeddings=True
        )[0].tolist()
        
        # Arama yap (query_points kullan)
        search_results = self.client.query_points(
            collection_name=self.collection_name,
            query=query_embedding,
            limit=limit,
            score_threshold=score_threshold
        )
        
        results = []
        for result in search_results.points:
            results.append({
                "score": result.score,
                "payload": result.payload
            })
        
        return results

    def advanced_search(self, query: str, filters: Dict = None, limit: int = 5) -> List[Dict[str, Any]]:
        """Gelişmiş filtreleme ile arama"""
        query_embedding = self.model.encode(
            [query], 
            convert_to_numpy=True,
            normalize_embeddings=True
        )[0].tolist()
        
        # Filter oluştur
        filter_conditions = None
        if filters:
            conditions = []
            
            if 'daire' in filters:
                conditions.append(models.FieldCondition(
                    key="daire",
                    match=models.MatchValue(value=filters['daire'])
                ))
            
            if 'karar_turu' in filters:
                conditions.append(models.FieldCondition(
                    key="karar_turu", 
                    match=models.MatchValue(value=filters['karar_turu'])
                ))
            
            if 'year' in filters:
                conditions.append(models.FieldCondition(
                    key="dates",
                    match=models.MatchText(text=str(filters['year']))
                ))
            
            if conditions:
                filter_conditions = models.Filter(must=conditions)
        
        # query_points kullan
        search_results = self.client.query_points(
            collection_name=self.collection_name,
            query=query_embedding,
            query_filter=filter_conditions,
            limit=limit
        )
        
        results = []
        for result in search_results.points:
            results.append({
                "score": result.score,
                "payload": result.payload
            })
        
        return results

    def get_collection_info(self):
        """Collection bilgilerini getir"""
        try:
            info = self.client.get_collection(collection_name=self.collection_name)
            result = {
                "status": str(info.status),
                "vectors_count": info.vectors_count if hasattr(info, 'vectors_count') else 0,
            }
            
            # Mevcut attributeları kontrol et ve ekle
            if hasattr(info, 'segments_count'):
                result["segments_count"] = info.segments_count
            if hasattr(info, 'indexed_vectors_count'):
                result["indexed_vectors_count"] = info.indexed_vectors_count
            if hasattr(info, 'points_count'):
                result["points_count"] = info.points_count
                
            return result
        except Exception as e:
            return {"error": str(e)}

In [5]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision

# ENV dosyasını yükle
dotenv_path = "/home/yapayzeka/ahsen_bulbul/qdrant.env"
load_dotenv(dotenv_path=dotenv_path)

openai_api_key = os.getenv("OPENAI_API_KEY")
qdrant_url = os.getenv("QDRANT_URL")
api_key = os.getenv("QDRANT_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY bulunamadı. qdrant.env dosyasını kontrol et.")


class RAGEvaluator:
    def __init__(self, csv_path: str, sample_size: int = 5):
        """
        CSV'den verileri yükleyen ve RAG değerlendirmesi yapan sınıf.
        """
        self.csv_path = csv_path
        self.sample_size = sample_size
        self.df = None
        self.dataset = None

    def load_csv(self):
        """CSV dosyasını oku ve gerekli sütunları hazırla"""
        print(f"CSV okunuyor: {self.csv_path}")
        self.df = pd.read_csv(self.csv_path)
        print(f"Toplam satır: {len(self.df)}")

        # Context sütunu oluştur
        self.df["contexts"] = self.df.apply(
            lambda row: [
                f"Daire: {row['daire']}",
                f"Mahkeme: {row['mahkeme']}",
                f"Karar Türü: {row['karar_turu']}"
            ],
            axis=1
        )

        # question ve ground_truth sütunları oluştur
        self.df["question"] = self.df["chunk_text"]  # şimdilik chunk_text'i soru gibi kullandık
        self.df["ground_truth"] = self.df["esasNo"].astype(str)  # veya kararNo

        # sadece küçük bir sample al (API kotasını aşmamak için)
        if self.sample_size and len(self.df) > self.sample_size:
            self.df = self.df.sample(self.sample_size, random_state=42)
            print(f"Sample alındı: {len(self.df)} satır")

    def prepare_dataset(self):
        """Pandas DataFrame'i HuggingFace Dataset formatına çevir"""
        if self.df is None:
            raise ValueError("CSV yüklenmemiş. Önce load_csv() çağır.")
        
        self.dataset = Dataset.from_pandas(
            self.df[["question", "chunk_text", "contexts", "ground_truth"]].rename(
                columns={"chunk_text": "answer"}
            )
        )
        print("Dataset hazır.")

    def evaluate_rag(self):
        """RAG değerlendirmesini çalıştır"""
        if self.dataset is None:
            raise ValueError("Dataset hazırlanmadı. Önce prepare_dataset() çağır.")
        
        print("RAG değerlendirmesi başlatılıyor...")
        results = evaluate(
            self.dataset,
            metrics=[faithfulness, answer_relevancy, context_precision, context_recall]
        )
        return results

    def print_evaluation_results(self, results):
        """Evaluation sonuçlarını yazdır"""
        print("\n=== RAG Değerlendirme Sonuçları ===")
        
        try:
            # RAGAS EvaluationResult nesnesinden metrikleri al
            if hasattr(results, 'to_pandas'):
                # Pandas DataFrame'e çevir
                df_results = results.to_pandas()
                for column in df_results.columns:
                    if df_results[column].dtype in ['float64', 'float32']:
                        mean_score = df_results[column].mean()
                        print(f"{column}: {mean_score:.4f}")
            
            elif hasattr(results, '__dict__'):
                # Nesne özelliklerine direkt erişim
                for metric, score in results.__dict__.items():
                    if isinstance(score, (int, float)):
                        print(f"{metric}: {score:.4f}")
            
            else:
                # Manuel olarak bilinen metrikleri yazdır
                metric_names = ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall']
                for metric in metric_names:
                    if hasattr(results, metric):
                        score = getattr(results, metric)
                        print(f"{metric}: {score:.4f}")
                        
        except Exception as e:
            print(f"Sonuçları yazdırırken hata: {e}")
            print("Results objesi tipi:", type(results))
            print("Results objesi:", results)


# Basit LegalDocumentVectorDB mock sınıfı (gerçek implementasyon için ayrı dosyada olmalı)
class LegalDocumentVectorDB:
    def __init__(self, qdrant_url, api_key, collection_name):
        self.qdrant_url = qdrant_url
        self.api_key = api_key
        self.collection_name = collection_name
        print(f"LegalDocumentVectorDB initialized for collection: {collection_name}")

    def create_collection(self, recreate=False):
        print(f"Collection {'recreated' if recreate else 'created'}: {self.collection_name}")

    def process_csv(self, csv_path):
        print(f"Processing CSV: {csv_path}")
        # Mock implementation - gerçekte CSV'yi işleyip DataFrame döndürür
        return pd.read_csv(csv_path) if os.path.exists(csv_path) else None

    def upload_to_qdrant(self, df, batch_size=50):
        print(f"Uploading {len(df)} records to Qdrant in batches of {batch_size}")

    def get_collection_info(self):
        return {
            "collection_name": self.collection_name,
            "status": "active",
            "vector_count": 1000  # mock data
        }

    def search(self, query, limit=5, score_threshold=0.5):
        # Mock search results
        return [
            {
                "score": 0.85,
                "payload": {
                    "daire": "4. Hukuk Dairesi",
                    "esas_no": "2023/1234",
                    "karar_no": "2023/5678",
                    "chunk_text": "İhtiyati tedbir kararının tazminat yükümlülüğü hakkında..."
                }
            }
        ] * min(limit, 3)  # Mock olarak 3 sonuç döndür


def main():
    CSV_FILE = "/home/yapayzeka/ahsen_bulbul/model/langchain/recursive/yargitay_chunks.csv"

    # CSV dosyasının varlığını kontrol et
    if not os.path.exists(CSV_FILE):
        print(f"UYARI: CSV dosyası bulunamadı: {CSV_FILE}")
        print("Mock data ile devam ediliyor...")
        # Mock CSV oluştur (test için)
        mock_data = {
            'daire': ['4. Hukuk Dairesi'] * 5,
            'mahkeme': ['Yargıtay'] * 5,
            'karar_turu': ['Temyiz'] * 5,
            'chunk_text': [
                'İhtiyati tedbir kararının tazminat yükümlülüğü konusunda...',
                'Sözleşmeli fesih durumunda tazminat hesaplanması...',
                'Manevi tazminat miktarının belirlenmesinde...',
                'İş kazası sonucu maddi tazminat talebinin...',
                'Tecavüz fiili nedeniyle manevi tazminat...'
            ],
            'esasNo': ['2023/1234', '2023/1235', '2023/1236', '2023/1237', '2023/1238'],
            'kararNo': ['2023/5678', '2023/5679', '2023/5680', '2023/5681', '2023/5682']
        }
        
        # Geçici CSV oluştur
        temp_csv = "/tmp/mock_yargitay_chunks.csv"
        pd.DataFrame(mock_data).to_csv(temp_csv, index=False)
        CSV_FILE = temp_csv

    # Qdrant tarafı
    db = LegalDocumentVectorDB(
        qdrant_url=qdrant_url,
        api_key=api_key,
        collection_name="hukuki_kararlar"
    )
    db.create_collection(recreate=True)
    df = db.process_csv(CSV_FILE)
    if df is not None:
        db.upload_to_qdrant(df, batch_size=50)

    # Collection bilgilerini göster
    info = db.get_collection_info()
    print("\n=== Qdrant Collection Bilgileri ===")
    print(json.dumps(info, indent=2, ensure_ascii=False))

    # Örnek aramalar
    print("\n=== Örnek Aramalar ===")
    results = db.search("ihtiyati tedbir tazminat", limit=3, score_threshold=0.6)
    for i, result in enumerate(results, 1):
        print(f"\n{i}. Sonuç (Skor: {result['score']:.3f})")
        print(f"   Daire: {result['payload']['daire']}")
        print(f"   Esas No: {result['payload']['esas_no']}")
        print(f"   Karar No: {result['payload']['karar_no']}")
        print(f"   Metin: {result['payload']['chunk_text'][:200]}...")

    # RAG Değerlendirmesi
    try:
        evaluator = RAGEvaluator(CSV_FILE, sample_size=5)
        evaluator.load_csv()
        evaluator.prepare_dataset()
        rag_results = evaluator.evaluate_rag()
        
        # Sonuçları yazdır (düzeltilmiş versiyon)
        evaluator.print_evaluation_results(rag_results)
        
    except Exception as e:
        print(f"RAG değerlendirmesinde hata oluştu: {e}")
        print("Bu genellikle OPENAI_API_KEY eksikliği veya API limiti aşımından kaynaklanır.")


if __name__ == "__main__":
    main()

LegalDocumentVectorDB initialized for collection: hukuki_kararlar
Collection recreated: hukuki_kararlar
Processing CSV: /home/yapayzeka/ahsen_bulbul/model/langchain/recursive/yargitay_chunks.csv
Uploading 137 records to Qdrant in batches of 50

=== Qdrant Collection Bilgileri ===
{
  "collection_name": "hukuki_kararlar",
  "status": "active",
  "vector_count": 1000
}

=== Örnek Aramalar ===

1. Sonuç (Skor: 0.850)
   Daire: 4. Hukuk Dairesi
   Esas No: 2023/1234
   Karar No: 2023/5678
   Metin: İhtiyati tedbir kararının tazminat yükümlülüğü hakkında......

2. Sonuç (Skor: 0.850)
   Daire: 4. Hukuk Dairesi
   Esas No: 2023/1234
   Karar No: 2023/5678
   Metin: İhtiyati tedbir kararının tazminat yükümlülüğü hakkında......

3. Sonuç (Skor: 0.850)
   Daire: 4. Hukuk Dairesi
   Esas No: 2023/1234
   Karar No: 2023/5678
   Metin: İhtiyati tedbir kararının tazminat yükümlülüğü hakkında......
CSV okunuyor: /home/yapayzeka/ahsen_bulbul/model/langchain/recursive/yargitay_chunks.csv
Toplam satır:

Evaluating:  80%|████████  | 16/20 [02:56<00:22,  5.52s/it]Exception raised in Job[11]: TimeoutError()
Exception raised in Job[10]: TimeoutError()
Evaluating: 100%|██████████| 20/20 [04:58<00:00, 14.91s/it]



=== RAG Değerlendirme Sonuçları ===
faithfulness: nan
answer_relevancy: nan
context_precision: nan
context_recall: nan
